In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Getting Started with Grounding with Gemini in Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fgrounding%2Fintro-grounding-gemini.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/grounding/intro-grounding-gemini.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

| | |
|-|-|
|Author(s) | [Holt Skinner](https://github.com/holtskinner), [Kristopher Overholt](https://github.com/koverholt) |

**_NOTE_**: このノートブックは以下の環境でテストされています:

* Python version = 3.11

## 概要

[Vertex AI のグラウンディング](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini) を使用すると、生成テキスト モデルを使用して、独自のドキュメントやデータに基づいたコンテンツを生成できます。この機能により、モデルは実行時にトレーニング データを超える情報にアクセスできます。モデルの応答を Google 検索結果または [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction) 内のデータ ストアにグラウンディングすることで、データに基づいた LLM はより正確で最新の関連性の高い応答を生成できます。

グラウンディングには、次の利点があります。

- モデルの幻覚（モデルが事実に基づかないコンテンツを生成するインスタンス）を削減します。
- モデルの応答を特定の情報、ドキュメント、およびデータ ソースに固定します。
- 生成されたコンテンツの信頼性、精度、および適用性を強化します。

Vertex AI のグラウンディングのコンテキストでは、2 つの異なるグラウンディング ソースを構成できます。

1. 公開され、インデックス付けされているデータの Google 検索結果
2. [Vertex AI Search のデータ ストア](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest)。これには、ウェブサイト データ、非構造化データ、または構造化データの形式で独自のデータを含めることができます。

### 許可リスト

このサンプル ノートブックの一部の機能では、許可リストを介して特定の機能にアクセスする必要があります。 [Vertex AI Search を使用した Grounding](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/ground-gemini) はパブリック プレビューで利用できますが、Google 検索結果を使用した Grounding は一般公開されています。

このサービスを本番環境アプリケーションで使用する場合は、[Google 検索エントリ ポイントを使用する](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/grounding-search-entry-points) も必要になります。

### 目的

このチュートリアルでは、次の方法を学習します。

- Google 検索結果に基づいた LLM テキストおよびチャット モデル応答を生成する
- 非グラウンディング LLM 応答とグラウンディング LLM 応答の結果を比較する
- Vertex AI Search でデータ ストアを作成して使用し、カスタム ドキュメントとデータに応答をグラウンディングする
- Vertex AI Search の結果に基づいた LLM テキストおよびチャット モデル応答を生成する

このチュートリアルでは、次の Google Cloud AI サービスとリソースを使用します。

- Vertex AI
- Vertex AI Search and Conversation

実行する手順は次のとおりです。

- LLM を構成し、さまざまな例のプロンプトを送信する
- Vertex AI の生成テキストおよびチャット モデルに例のプロンプトを送信する
- 独自のデータを使用して Vertex AI Search にデータ ストアを設定する
- さまざまなレベルのグラウンディング (グラウンディングなし、ウェブ グラウンディング、データ ストア グラウンディング) で例のプロンプトを送信する

## 始める前に

### Google Cloud プロジェクトをセットアップする

**ノートブックの環境に関係なく、次の手順が必要です。**

1. [Google Cloud プロジェクトを選択または作成します](https://console.cloud.google.com/cloud-resource-manager)。初めてアカウントを作成すると、コンピューティング/ストレージ費用に対して 300 ドルの無料クレジットが付与されます。

1. [プロジェクトで課金が有効になっていることを確認します](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [Vertex AI および Vertex AI Agent Builder API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,discoveryengine.googleapis.com) を有効にします。

1. このノートブックをローカルで実行している場合は、[Cloud SDK](https://cloud.google.com/sdk) をインストールする必要があります。

### インストール

このノートブックを実行するために必要な次のパッケージをインストールします。

In [1]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


パッケージをインストールした後、カーネルを再起動します。

In [2]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

<div class="alert alert-block alert-warning">
<b>⚠️ カーネルが再起動します。次のステップに進む前に、完了するまでお待ちください。 ⚠️</b>
</div>

### Google Cloud アカウントを認証する

このノートブックを Google Colab で実行している場合は、環境を認証する必要があります。これを行うには、以下の新しいセルを実行します。Vertex AI Workbench を使用している場合、この手順は必要ありません。

In [3]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### Google Cloud プロジェクト情報を設定し、Vertex AI SDK を初期化します

Vertex AI の使用を開始するには、既存の Google Cloud プロジェクトがあり、[Vertex AI API を有効にする](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) 必要があります。

[プロジェクトと開発環境の設定](https://cloud.google.com/vertex-ai/docs/start/cloud-environment) について詳しくは、こちらをご覧ください。

**プロジェクト ID がわからない場合** は、次の操作を試してください。
* `gcloud config list` を実行します。
* `gcloud projects list` を実行します。
* サポート ページをご覧ください: [プロジェクト ID を見つける](https://support.google.com/googleapi/answer/7014113)

Vertex AI で使用される `REGION` 変数を変更することもできます。 [Vertex AI リージョン](https://cloud.google.com/vertex-ai/docs/general/locations) の詳細をご覧ください。

In [4]:
PROJECT_ID = "YOUR_PROJECT_ID"  # @param {type:"string"}
REGION = "YOUR_REGION"  # @param {type: "string"}

In [5]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

/Users/yamamoto.akihiro/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


ValueError: Unsupported region for Vertex AI, select from frozenset({'europe-west6', 'asia-northeast3', 'us-west2', 'us-east4', 'us-west3', 'australia-southeast2', 'europe-west8', 'europe-southwest1', 'us-west1', 'me-central1', 'asia-northeast2', 'europe-north1', 'europe-central2', 'me-central2', 'europe-west4', 'us-south1', 'asia-south1', 'australia-southeast1', 'me-west1', 'asia-east1', 'us-east1', 'asia-southeast2', 'asia-east2', 'europe-west1', 'africa-south1', 'asia-southeast1', 'asia-northeast1', 'us-central1', 'southamerica-west1', 'europe-west2', 'northamerica-northeast2', 'southamerica-east1', 'us-east5', 'europe-west12', 'europe-west3', 'us-west4', 'europe-west9', 'northamerica-northeast1'})

: 

### Import libraries

In [ ]:
from IPython.display import Markdown, display
from vertexai.generative_models import (
    GenerationResponse,
    GenerativeModel,
    Tool,
    grounding,
)
from vertexai.preview.generative_models import grounding as preview_grounding

Vertex AIからGeminiモデルを初期化する:

In [ ]:
model = GenerativeModel("gemini-1.5-pro")

## Helper functions

In [ ]:
def print_grounding_response(response: GenerationResponse):
    """Prints Gemini response with grounding citations."""
    grounding_metadata = response.candidates[0].grounding_metadata

    # Citation indices are in byte units
    ENCODING = "utf-8"
    text_bytes = response.text.encode(ENCODING)

    prev_index = 0
    markdown_text = ""

    for grounding_support in grounding_metadata.grounding_supports:
        text_segment = text_bytes[
            prev_index : grounding_support.segment.end_index
        ].decode(ENCODING)

        footnotes_text = ""
        for grounding_chunk_index in grounding_support.grounding_chunk_indices:
            footnotes_text += f"[{grounding_chunk_index + 1}]"

        markdown_text += f"{text_segment} {footnotes_text}\n"
        prev_index = grounding_support.segment.end_index

    if prev_index < len(text_bytes):
        markdown_text += str(text_bytes[prev_index:], encoding=ENCODING)

    markdown_text += "\n----\n## Grounding Sources\n"

    if grounding_metadata.web_search_queries:
        markdown_text += (
            f"\n**Web Search Queries:** {grounding_metadata.web_search_queries}\n"
        )
        if grounding_metadata.search_entry_point:
            markdown_text += f"\n**Search Entry Point:**\n {grounding_metadata.search_entry_point.rendered_content}\n"
    elif grounding_metadata.retrieval_queries:
        markdown_text += (
            f"\n**Retrieval Queries:** {grounding_metadata.retrieval_queries}\n"
        )

    markdown_text += "### Grounding Chunks\n"

    for index, grounding_chunk in enumerate(
        grounding_metadata.grounding_chunks, start=1
    ):
        context = grounding_chunk.web or grounding_chunk.retrieved_context
        if not context:
            print(f"Skipping Grounding Chunk {grounding_chunk}")
            continue

        markdown_text += f"{index}. [{context.title}]({context.uri})\n"

    display(Markdown(markdown_text))

## 例: Google 検索結果でのグラウンディング

この例では、グラウンディングなしの LLM 応答と、Google 検索の結果でグラウンディングされた応答を比較します。Google ストアの最近のハードウェア リリースについて質問します。

In [ ]:
PROMPT = "あなたは天文学の専門家です。アメリカで次に日食が起こるのはいつですか?"

### グラウンディングなしのテキスト生成

グラウンディングなしで LLM に予測リクエストを送信します。

In [ ]:
response = model.generate_content(PROMPT)

display(Markdown(response.text))

### Google 検索結果に基づいたテキスト生成

これで、`grounding.GoogleSearchRetrieval()` の `grounding` ツールで `tools` キーワード引数を追加して、最初にプロンプ​​トを使用して Google 検索を実行し、次に Web 検索結果に基づいて回答を作成するように LLM に指示できます。

検索クエリと [検索エントリ ポイント](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/grounding-search-entry-points) は、応答内の各 `Candidate` で使用できます。ヘルパー関数 `print_grounding_response()` は、引用を含む応答テキストを出力します。

In [ ]:
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = model.generate_content(PROMPT, tools=[tool])

print_grounding_response(response)

グラウンディングなしの応答には、LLM からの日食に関する限定された情報のみが含まれていることに注意してください。一方、Web 検索結果にグラウンディングされた応答には、グラウンディングありの LLM リクエストの一部として返される Web 検索結果からの最新情報が含まれています。

## 例: カスタム ドキュメントとデータによるグラウンディング

この例では、グラウンディングなしの LLM 応答と、[Vertex AI Search のデータ ストアの結果](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest) にグラウンディングされた応答を比較します。[BigQuery のオブジェクト テーブル](https://cloud.google.com/bigquery/docs/object-table-introduction) を作成するための GoogleSQL クエリについて質問します。

### Vertex AI Search でデータ ストアを作成する

この例では、ドキュメントを含む Google Cloud ウェブサイトのコンテンツを含むウェブサイト ベースのデータ ストアを使用します。

ラボの手順マニュアルの __タスク 4__ と __タスク 5__ を参照して、以下の手順を実行してください。データ ストアを作成したら、下のセルに入力する ID をメモしてください。

Vertex AI Search ドキュメントのチュートリアルの手順に従って、次の操作を行います。

1. [**含めるサイト**] フィールドで URL パターン `cloud.google.com/*` を使用する [ウェブサイト データを含むデータ ストアを作成](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_data_store)。
2. そのデータ ストアにアタッチされた [検索アプリを作成](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app)。また、データ ストア内のインデックス付きレコードを検索できるように、**Enterprise エディションの機能** も有効にする必要があります。

注: データ ストアは、Gemini で使用しているプロジェクトと同じプロジェクト内にある必要があります。

__データ ストアを作成したら、データ ストア ID を取得して以下に入力します。__

注: グラウンディングでデータ ストアを使用する前に、データの取り込みが完了するまで待つ必要があります。詳細については、[データ ストアの作成](https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es) をご覧ください。

In [ ]:
DATA_STORE_PROJECT_ID = PROJECT_ID  # @param {type:"string"}
DATA_STORE_REGION = "global"  # @param {type:"string"}
# Replace this with your data store ID from Vertex AI Search
DATA_STORE_ID = "your-data-store-id_1234567890123"  # @param {type:"string"}

ここで、BigQuery のオブジェクト テーブルとその使用タイミングについて質問してみましょう。

In [ ]:
PROMPT = "BigQuery でオブジェクト テーブルを使用するのはいつですか? また、データはどのように保存されますか?"

### グラウンディングなしのテキスト生成

グラウンディングなしで LLM に予測リクエストを送信します。

In [ ]:
response = model.generate_content(PROMPT)

display(Markdown(response.text))

### Vertex AI 検索結果に基づいたテキスト生成

これで、`tools` キーワード引数に `grounding.VertexAISearch()` というグラウンディング ツールを追加して、最初にカスタム データ ストア内で検索を実行し、次に関連ドキュメントに基づいて回答を作成するように LLM に指示できます。

In [ ]:
tool = Tool.from_retrieval(
    preview_grounding.Retrieval(
        preview_grounding.VertexAISearch(
            datastore=DATA_STORE_ID,
            project=DATA_STORE_PROJECT_ID,
            location=DATA_STORE_REGION,
        )
    )
)

response = model.generate_content(PROMPT, tools=[tool])
print_grounding_response(response)

根拠のない応答には、BigQuery のオブジェクト テーブルに関する LLM からの限られた情報しか含まれず、正確ではない可能性があることに注意してください。一方、Vertex AI Search の結果に根拠のある応答には、BigQuery に関する Google Cloud ドキュメントの最新情報と、その情報の引用が含まれています。

<div class="alert alert-block alert-warning">
<b>⚠️ Important notes:</b><br>
<br>
<b>前のセルを実行したときにエラーが発生した場合:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;このサンプルノートブックをVertex AI Searchのデータストアで動作させるには,<br>
&nbsp;&nbsp;&nbsp;&nbsp;<a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_data_store">data store</a>を作成する必要があります。 <b>それと</b> Vertex AI Searchの<a href="https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app">search app</a> を有効にしてください。<br>
&nbsp;&nbsp;&nbsp;&nbsp;data storeのみを作成すると、data storeに対してクエリを実行するときに、前の要求によってエラーが返されます。
<br><br>
<b>前のセルを実行したときに空の応答が返された場合:</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;グラウンディング付きデータ ストアを使用する前に、データの取り込みが完了するまで待つ必要があります。<br>
&nbsp;&nbsp;&nbsp;&nbsp;詳細についてはこちらを参照してください <a href="https://cloud.google.com/generative-ai-app-builder/docs/create-data-store-es">create a data store</a>.
</div>
</div>

## 例: グラウンディングされたチャット応答

Vertex AI でチャット モデルを操作するときにもグラウンディングを使用できます。この例では、グラウンディングのない LLM 応答と、Google 検索の結果と Vertex AI Search のデータ ストアにグラウンディングされた応答を比較します。

Vertex AI に関する質問をし、続いて Vertex AI で管理されているデータセットに関する質問をします。

In [ ]:
PROMPT = "Vertex AI の管理対象データセットとは何ですか?"
PROMPT_FOLLOWUP = "どのような種類のデータを使用できますか?"

### 接地なしのチャット セッション

チャット セッションを開始し、接地なしで LLM にメッセージを送信します。

In [ ]:
chat = model.start_chat()

response = chat.send_message(PROMPT)
display(Markdown(response.text))

response = chat.send_message(PROMPT_FOLLOWUP)
display(Markdown(response.text))

### Google 検索結果に基づいたチャット セッション

これで、`tools` キーワード引数に `grounding.GoogleSearchRetrieval()` というグラウンディング ツールを追加して、チャット モデルに最初にプロンプ​​トを使用して Google 検索を実行し、次に Web 検索結果に基づいて回答を作成するように指示できます。

In [ ]:
chat = model.start_chat()
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = chat.send_message(PROMPT, tools=[tool])
print_grounding_response(response)

response = chat.send_message(PROMPT_FOLLOWUP, tools=[tool])
print_grounding_response(response)

### Vertex AI 検索結果に基づいたチャット セッション

これで、`tools` キーワード引数と `grounding.VertexAISearch()` というグラウンディング ツールを追加して、チャット モデルに最初にカスタム データ ストア内で検索を実行し、次に関連ドキュメントに基づいて回答を作成するように指示できます。

In [ ]:
chat = model.start_chat()
tool = Tool.from_retrieval(
    preview_grounding.Retrieval(
        preview_grounding.VertexAISearch(
            datastore=DATA_STORE_ID,
            project=DATA_STORE_PROJECT_ID,
            location=DATA_STORE_REGION,
        )
    )
)
response = chat.send_message(PROMPT, tools=[tool])
print_grounding_response(response)

response = chat.send_message(PROMPT_FOLLOWUP, tools=[tool])
print_grounding_response(response)

## クリーンアップ

このノートブックで使用されているリソースに対して Google Cloud アカウントに課金されないようにするには、次の手順に従います。

1. 不要な Google Cloud 課金を回避するには、[Google Cloud コンソール](https://console.cloud.google.com/) を使用して、不要なプロジェクトを削除します。詳細については、[プロジェクトの管理と削除](https://cloud.google.com/resource-manager/docs/creating-managing-projects) に関する Google Cloud ドキュメントをご覧ください。

1. 既存の Google Cloud プロジェクトを使用した場合は、アカウントに課金されないように、作成したリソースを削除します。詳細については、[Vertex AI Search のデータストアからデータを削除する](https://cloud.google.com/generative-ai-app-builder/docs/delete-datastores) に関するドキュメントを参照してから、データストアを削除してください。
1. Google Cloud Console で [Vertex AI Search and Conversation API](https://console.cloud.google.com/apis/api/discoveryengine.googleapis.com) と [Vertex AI API](https://console.cloud.google.com/apis/api/aiplatform.googleapis.com) を無効にします。